In [1]:
import pandas as pd

In [8]:
df_test = pd.read_csv('dataset/data_test.csv')

In [11]:
df_train = pd.read_csv('dataset/data_train.csv')

In [22]:
df_etiquetas = pd.read_csv('dataset/Etiquetas.csv')

In [10]:
df_test.shape

(555, 8)

In [14]:
df_train.shape

(850, 8)

In [19]:
df_train.head()

,Id,Cultivo,Longitud,Latitud,Elevacion,Dataset,Campania,GlobalId
0,1,S,-62.144163,-33.800202,104.111862,BC,18/19,1
1,4,M,-62.155418,-33.801742,105.698082,BC,18/19,4
2,6,N,-62.163615,-33.808092,104.233162,BC,18/19,6
3,7,M,-62.164773,-33.813671,103.859932,BC,18/19,7
4,9,M,-62.097200,-33.778628,98.532104,BC,18/19,9


In [29]:
df_train['Campania'].unique()

array(['18/19', '19/20'], dtype=object)

In [15]:
df_train.describe()

,Id,Longitud,Latitud,Elevacion,GlobalId
count,850.000000,850.000000,850.000000,850.000000,850.000000
mean,243.943529,-61.905162,-33.800339,102.923320,750.588235
std,144.458394,0.298277,0.184910,13.791425,418.263748
min,1.000000,-62.861688,-34.375327,-0.000023,1.000000
25%,119.000000,-62.099017,-33.872251,99.859977,411.500000
50%,235.000000,-61.920053,-33.798076,103.869263,758.500000
75%,366.750000,-61.709385,-33.671438,109.479976,1108.500000
max,550.000000,-61.210180,-33.458219,126.779977,1455.000000


In [17]:
df_train['Dataset'].unique()

array(['BC', 'BCR'], dtype=object)

In [23]:
df_etiquetas.head()

,CultivoId,Cultivo,Tipo
0,1,S,SOJA 1
1,2,s,SOJA 2
2,3,M,MAIZ TEMP
3,4,m,MAIZ TARD O 2DA
4,5,T,TRIGO


In [25]:
df_etiquetas['CultivoId'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])

In [31]:
import ee

In [32]:
ee.Authenticate()

Enter verification code:  4/5wE6tjuX0i7B22LCeDODpYZfvTKZNjQAcOLfS3UNdMVJOnSPF5kcFLM



Successfully saved authorization token.


In [34]:
ee.Initialize()

In [35]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [36]:
# Import the Image function from the IPython.display module. 
from IPython.display import Image

# Display a thumbnail of global elevation.
Image(url = dem.updateMask(dem.gt(0))
  .getThumbURL({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [37]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3, height=500)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

ModuleNotFoundError: No module named 'folium'

In [38]:
# Import the matplotlib.pyplot module.
import matplotlib.pyplot as plt

# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()

<Figure size 640x480 with 1 Axes>